# __Table Of Contents__

1. [Problem Context and Definition](#p-c-and-d)

## Initial Model
2. [Initial MIP Formulation](#initial-model)
3. [Initial Gurobi Code]
4. [Initial Results]

## Improvements
### Phase One
5. [Idea 1 - Implicit Continuous Variables]
6. [Idea 2 - Binary Representation of Element Sections]
7. [Idea 3 - Lazy Callbacks of Grouping Constraint]
8. [Results of Phase One Improvements]

### Phase Two
9. [Idea 4 - Partial Symmetry Breaking]
10. [Idea 5 - Clique Constraints]
11. [Results of Phase Two Improvements]

12. [Contribution Declratation]

# **Problem Context and Definition**

Designers can choose different structural sections for each element.

The higher the loads, the larger the section needed, the higher the material and labour cost to buy and install that element.

Generally, designers choose the smallest section necessary to fulfill the requirements.

In a typical building, there can be thousands of individual elements.

It would be unreasonable to design each one individually.

So: engineers group elements with similar conditions and design based on the “worst-case” in each group.

Each element has a ‘minimum viable section'.

The grouped section must be larger/equal to this minimum.

An element must have a larger/equal section to the element below.

Section defined as integer with associated cost (larger number = larger associated cost).

*  Groups are defined as acting over:

*  A certain subset of columns.

*  A certain range of floors.

*  Every element within that set must exist and must be in the group.

*  If a group is defined over a column/floor, an element has to exist there and must be in the group.

# __Initial Model__

### **Sets**

For our initial model we define the following sets:

  * Levels: $$L={1,...,n_{L}}$$
  * Columns: $$C={1,...,n_{C}}$$
  * Sections: $$S={1,...,n_{S}}$$
  * Groups: $$C={1,...,n_{G}}$$

We define the following heuristic to obtain an upperbound for the number of group:
$$n_{G}=min(100,\sum\limits_{c ϵ C}|S_{c}|, \sum\limits_{l ϵ L}|S_{l}|)$$



### **Constants**

Based on our input data we define the following constants:

  * Minmimum Viable Section:
  $$mvs_{cl}=
  \left\{
  \begin{array}{l}
  s\hspace{1cm}\text{if element exists} \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ g\ ϵ\ G
  $$

  * Whether an element exists:
  $$e_{cl}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if element exists} \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ g\ ϵ\ G
  $$

  * Cost of Section:
  $$cs_{s}\ ∀\ s\ ϵ\ S$$

  * Costper Group:$$cg$$

  * Big M: $$M_{L}=n_{L}+1,\ M_{S}=n_{S}+1$$


### **Decision Variables**

The following are the decision variables:
  * Variables relating to grouping:
  $$x_{clg}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if element at column}\ c\ \text{, level}\ l\ \text{is in group g} \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ g\ ϵ\ G
  $$

  $$ge_{g}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if group}\ g\ \text{exists} \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ g\ ϵ\ G
  $$

  $$cig_{cg}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if column}\ c\ \text{is in group}\ g \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ g\ ϵ\ G
  $$

  $$lig_{lg}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if level}\ l\ \text{is in group}\  g \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ l\ ϵ\ L,\ g\ ϵ\ G
  $$

  * Variables relating to levels:
  $$glb_{g}= \text{The smallest level in group}\ g\hspace{1cm} 0\leq glb_{g}\leq n_{l} \hspace{1cm}∀\ g\ ϵ\ G,\ glb_g\ ϵ\ Z$$

  $$gub_{g}= \text{The highest level in group}\ g\hspace{1cm} 0\leq glb_{g}\leq n_{l} \hspace{1cm}∀\ g\ ϵ\ G,\ glb_g\ ϵ\ Z$$

  $$zu_{lg}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if}\ gub_{g}\geq\ l  \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ g\ ϵ\ G
  $$
  
  
  $$zl_{lg}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if}\ gub_{g}\leq\ l  \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ g\ ϵ\ G
  $$

  * Variables relating to sections:

  $$gs_{g}= \text{Section of group}\ g\hspace{1cm} 0\leq gs_{g}\leq n_{g} \hspace{1cm}∀\ g\ ϵ\ G,\ gs_g\ ϵ\ Z$$
  
  $$es_{cl}= \text{Section of element at column}\ c\ \text{at level}\ l\hspace{1cm} 0\leq es_{cl}\leq n_{s} \hspace{1cm}∀\ c\ ϵ\ C,\ es_{cl}\ ϵ\ Z$$
  
  $$ec_{cl}= \text{Cost of element at column}\ c\ \text{at level}\ l\hspace{1cm} 0\leq es_{cl}\leq \max cs_{s} \hspace{1cm}∀\ c\ ϵ\ C,\ es_{cl}\ ϵ\ R$$

  $$esb_{cls}=
  \left\{
  \begin{array}{l}
  1\hspace{1cm}\text{if}\ es_{cl}= s  \\
  0\hspace{1cm}\text{otherwise}
  \end{array}
  \right.\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ s\ ϵ\ S
  $$

### **Objective Function**

$$min∑\limits_{c\ ϵ\ C,\ l\ ϵ\ L, g\ ϵ\ G}ec_{cl}+cg\times ge_{g}-gub_{g}+glb_g$$

The above ensures that we:
  1. Minimzie section cost
  2. Minimize number of groups
  3. Minimze upperbound and maximize the lowerbound

### **Constraints**

The following constraints are regarding grouping:
  * To ensure that each element that exists is in exactly one group:
  $$\sum\limits_{g\ ϵ\ G}x_{clg}=e_{cl}\hspace{1cm}∀\ c\ ϵ\ C, \ l\ ϵ\ L$$
  * Columns in group tied to element in group:
  $$cig_{cg}\geq x_{clg}\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ \epsilon \ L,\ g\ \epsilon\ G$$
  * Level in group tied to element in group:
  $$lig_{lg}\geq x_{clg}\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ \epsilon \ L,\ g\ \epsilon\ G$$
  * Group exists tied to element in group:
  $$ge_{g}\geq x_{clg}\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ \epsilon \ L,\ g\ \epsilon\ G$$

The following constraints are regarding sections:
  * One section per element:
  $$\sum\limits_{s\ ϵ\ S}ecb_{cls}=1\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L$$

  * Element section implies element cost:
  $$ecb_{cls}=1⇒es_{cl}=s\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ s\ ϵ\ S$$
  $$ecb_{cls}=1⇒ec_{cl}=s\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L,\ s\ ϵ\ S$$

  * Element section larger than minimum viable section, smaller than section below:
  $$es_{cl}\geq mvs_{cl}\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L$$
  $$es_{cl}\leq es_{c,\ l-1}\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L:\ e_{cl}=1\ \cap\ e_{c,l-1}=1$$

  * Element section linked with group section:
  $$es_{cl}\geq gs_g-M_s(1-x_{clg})\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ \epsilon \ L,\ g\ \epsilon\ G$$
  $$es_{cl}\leq gs_g+M_s(1-x_{clg})\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ \epsilon \ L,\ g\ \epsilon\ G$$

The following constraints are regarding levels:
  * Define group level UB and group level LB:
  $$l× lig_{lg}\leq gub_g\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$
  $$l× lig_{lg}+M_L(1-lig_{lg})\geq glb_g\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$

  * Define _ZU_ and _ZL_ (whether level is above lowerbound\below upperbound):
  $$M_L×zl_{lg}\geq l-glb_g+1\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$
  $$M_L×(1-zl_{lg})\geq glb_g-l\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$
  $$M_L×zu_{lg}\geq glb_g-l+1\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$
  $$M_L×(1-zu_{lg})\geq l-glb_g\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$

  * If level is below upperbound and above lowerbound, it is in the group:
  $$1+lig_{lg}\geq zu_{lg}+zl_{lg}\hspace{1cm}\forall\ l\ ϵ\ L,\ g\ ϵ\ G$$

  * If the column and level are in group, element at that level and column must be in the same group:
  $$cig_{cg}+lig_{lg}\leq 1+x_{}clg\hspace{1cm}\forall\ l\ ϵ\ L,\ c\ \epsilon\ C,\ g\ ϵ\ G$$

# **Phase One Improvements**

## **1. Implicit Continuous Variables**

When testing our model on small test data, we obsereved that gurobi tended to change a number of variables to be continuous, instead of integer or binary. As a result we changed the following variables to be contiuous instead of integer or binary:

$$ 0\leq glb_{g}\leq n_{l} \hspace{1cm}∀\ g\ ϵ\ G, \ glb_g\ ϵ\ R$$
$$ 0\leq glb_{g}\leq n_{l} \hspace{1cm}∀\ g\ ϵ\ G,\ glb_g\ ϵ\ R$$
$$zu_{lg}=[0,1]\hspace{1cm}∀\ c\ ϵ\ C,\ g\ ϵ\ G, zu_{lg}\ ϵ\ R$$
$$zl_{lg}=[0,1]\hspace{1cm}∀\ c\ ϵ\ C,\ g\ ϵ\ G, zu_{lg}\ ϵ\ R$$


## **2. Binary Model**
The element section integer variables can be easily represented by only binary variables by making some adjustments to our constraints:
  * We remove the following variables:
  $$es_{cl}$$
  $$ec_{cl}$$
  * And make the following adjustments to our constraints:
  $$\sum\limits_{s\ ϵ\ S}esb_{cls}=e_{cl}\hspace{1cm}\forall c\ ϵ\ C,\ l\ \epsilon\ L$$
  $$\sum\limits_{s\ ϵ\ S}s\times esb_{cls}\geq mvs_{cl}\hspace{1cm}\forall c\ ϵ\ C,\ l\ \epsilon\ L$$
  $$\sum\limits_{s\ ϵ\ S}s\times esb_{cls}\leq\sum\limits_{s\ ϵ\ S}s\times esb_{c,\ l-1,\ s}\hspace{1cm}∀\ c\ ϵ\ C,\ l\ ϵ\ L:\ e_{cl}=1\ \cap\ e_{c,l-1}=1$$
  $$\sum\limits_{s\ \epsilon\ S}s× esb_{cls}\geq gs_g-M_S(1-x_{clg})\hspace{1cm}∀\ c\ ϵ\ C,\ l\ \epsilon\ L,\ g\ ϵ\ G$$
  $$\sum\limits_{s\ \epsilon\ S}s× esb_{cls}\leq gs_g+M_S(1-x_{clg})\hspace{1cm}∀\ c\ ϵ\ C,\ l\ \epsilon\ L,\ g\ ϵ\ G$$
  * Finally we change our objective function to comply with the changes we have made:
  $$\sum\limits_{c\ \epsilon\ C,\ l\ ϵ\ L,\ g\ ϵ\ G}cs_s× ecb_{cls}+c_g×ge_g-gub_g+glb_g$$

## **3. Lazy Constraint**
We added the following constraint as a lazy callback constraint:
$$cig_{cg}+lig_{lg}\leq 1+x_{clg}\hspace{1cm}∀\ c\ \epsilon\ C,\ l\ ϵ\ L, g\ ϵ\ G$$

The reason for why we added this particular constraint as a lazy callback is that this is the most geometrically restricting constraint.

# **Gurobi Settings**

We have set a time limit of 30 minutes (1800 seconds) and memory limit of 10 GB.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import json
import psutil
import os
import sys
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import seaborn as sns

process = psutil.Process()

def get_heuristic_group_ub(columns):
  col_ub = 0
  level_ub = 0
  for col in columns:
    col_ub += len(set([elem for elem in col if elem > 0]))
  for lvl in range(len(columns[0])):
    level_ub += len(set([col[lvl] for col in columns if col[lvl] > 0]))
  return min(col_ub,level_ub), max(col_ub,level_ub)

# def heatmap(color_vals, annot_vals):
#   unique_values_1 = np.unique(color_vals)
#   unique_values_1 = unique_values_1[unique_values_1 > 0]

#   unique_values_2 = np.unique(annot_vals)
#   unique_values_2 = unique_values_2[unique_values_2 > 0]

#   palette_1 = sns.color_palette("viridis", len(unique_values_1))
#   palette_2 = sns.color_palette("viridis", len(unique_values_2))

#   color_dict_1 = {0: (1, 1, 1)}  # White for -1
#   for val, color in zip(unique_values_1, palette_1):
#       color_dict_1[val] = color

#   color_dict_2 = {0: (1, 1, 1)}  # White for 0
#   for val, color in zip(unique_values_2, palette_2):
#       color_dict_2[val] = color

#   sorted_vals_1 = np.sort(list(color_dict_1.keys()))
#   cmap_1 = mcolors.ListedColormap([color_dict_1[val] for val in sorted_vals_1])

#   sorted_vals_2 = np.sort(list(color_dict_2.keys()))
#   cmap_2 = mcolors.ListedColormap([color_dict_2[val] for val in sorted_vals_2])

#   bounds_1 = np.append(sorted_vals_1, sorted_vals_1[-1] + 1) - .5
#   norm_1 = mcolors.BoundaryNorm(bounds_1, cmap_1.N)

#   bounds_2 = np.append(sorted_vals_2, sorted_vals_2[-1] + 1) - .5
#   norm_2 = mcolors.BoundaryNorm(bounds_2, cmap_2.N)

#   fig, axes = plt.subplots(1,2,figsize=(10,5))

#   sns.heatmap(np.rot90(color_vals), annot=np.rot90(annot_vals), cmap=cmap_1, norm=norm_1,
#               linewidths=0.5, linecolor="black", cbar=False, ax=axes[0])

#   ax2 = sns.heatmap(np.rot90(annot_vals), annot=np.rot90(color_vals), cmap=cmap_2, norm=norm_2,
#                     linewidths=.5, linecolor="black", cbar=True, ax=axes[1])

#   # customizing the cbar
#   cbar = ax2.collections[0].colorbar
#   cbar.set_ticks(sorted_vals_2)
#   cbar.set_ticklabels(sorted_vals_2)

#   axes[0].set_title("Grouping of elements")
#   axes[1].set_title("Section of elements")

#   axes[0].axis("off")
#   axes[1].axis("off")

#   plt.tight_layout()
#   plt.show()

def lazy_callback(model, where):
    if where == GRB.Callback.MIPSOL:  # Check if a new solution is found
        # Retrieve the values of decision variables at the current solution
        column_vals = model.cbGetSolution(column_in_group)
        level_vals = model.cbGetSolution(level_in_group)
        x_vals = model.cbGetSolution(x)

        # Iterate through indices and add violated constraints
        for g in range(max_min_groups):
            for l in range(n_levels):
                for c in range(n_cols):
                    lhs = column_vals[g, c] + level_vals[g, l]  # Left-hand side
                    rhs = 1 + x_vals[g, c, l]  # Right-hand side
                    if lhs > rhs + 1e-6:  # Constraint is violated
                        model.cbLazy(column_in_group[g, c] + level_in_group[g, l] <= 1 + x[g, c, l])

if __name__ == "__main__":

  print("===INSTANCE START")


  script, instance, opt = sys.argv

  opt = int(opt)
  # instance = "4"
  # opt = "4"

  #opt: 1 - only int, 2 - cont, 4 - lazy,  6 - cont+lazy
  #opt: 3 - only bin, 5 - bin+cont, 7: bin+lazy, 8:bin+lazy+cont


  folderpath = os.getcwd()
  data_path = os.path.join(folderpath,"data.json")

  with open(data_path, 'r') as file:
    data = json.load(file)

  i = instance
  i_name = data[i]["name"]
  print(f"Instance Name: {instance}-{i_name}")
  test_data = data[i]["columns"]
  SectionCost = data[i]["section_costs"]
  n_cols = len(test_data)
  n_levels = len(test_data[0])

  max_min_groups, max_max_groups = get_heuristic_group_ub(test_data) #min(n_cols,n_levels)

  min_cost = sum([SectionCost[i] for col in test_data for i in col])

  GroupCost = round(min_cost/max_max_groups)
  max_min_groups = min(100, max_min_groups)
  n_cols = len(test_data)
  n_levels = len(test_data[0])

  M = n_levels + 2
  M_sections = max([max(i) for i in test_data]) + 1

  Xgcl = [(g,c,l) for l in range(n_levels) for c in range(n_cols) for g in range(max_min_groups)]
  Gs = [(g,s) for g in range(max_min_groups) for s in range(M_sections)]
  Es = [(c,l,s) for s in range(M_sections) for l in range(n_levels) for c in range(n_cols)]

  #c,l
  S = {(i,j): test_data[i][j] for j in range(n_levels) for i in range(n_cols)}
  S_bin = {(i,j,s): 1 if s >= test_data[i][j] else 0 for s in range(M_sections) for j in range(n_levels) for i in range(n_cols)}
  E = {(i,j): 1 if test_data[i][j] > 0 else 0 for j in range(n_levels) for i in range(n_cols)}


  # Create a new model
  model = gp.Model("Grouping-Optimization-int")

  # # Create variables

  # print("add vars")
  x = model.addVars(Xgcl, vtype = GRB.BINARY, name="x")

  # SectionCost = [0,100,400,800,1600,3200,6400,12800,14000]
  sections = set(i for i in range(len(SectionCost)))

  group_exists = model.addVars(max_min_groups, vtype = GRB.BINARY, name="group_exists")
  column_in_group = model.addVars(max_min_groups, n_cols, vtype = GRB.BINARY, name="col_in_group")
  level_in_group = model.addVars(max_min_groups, n_levels, vtype = GRB.BINARY, name="level_in_group")

  # can these be continuous? or must be int?

  #section size variables
  group_section = model.addVars(max_min_groups, vtype = GRB.INTEGER, name="group_section", lb = 0, ub = max(sections))
  element_section = model.addVars(S.keys(), vtype=GRB.INTEGER, name="element_section", lb=0, ub = max(sections))
  # binary variables y[i,j,k] that are 1 if element_section[i,j] == k
  y = model.addVars(S.keys(), range(len(sections)), vtype=GRB.BINARY, name="y")

  # cost variables for element_section
  cost_section = model.addVars(S.keys(), vtype=GRB.CONTINUOUS, name="cost_section")
  # Zs = model.addVars(Xgcl, vtype = GRB.BINARY, name = "Zs") #if group section = element section

  if opt in [2,5,6,8]:
    group_lower_bound = model.addVars(max_min_groups, vtype = GRB.CONTINUOUS, name="group_lb", lb = 0, ub = n_levels)
    group_upper_bound = model.addVars(max_min_groups, vtype = GRB.CONTINUOUS, name="group_ub", lb = 0, ub = n_levels)
    group_level_range = model.addVars(max_min_groups, vtype = GRB.CONTINUOUS, name="group_range", lb = 0, ub = n_levels)
    Zu = model.addVars(max_min_groups, n_levels, vtype = GRB.CONTINUOUS, name="Zu")
    Zl = model.addVars(max_min_groups, n_levels, vtype = GRB.CONTINUOUS, name="Zl")
  else:
    group_lower_bound = model.addVars(max_min_groups, vtype = GRB.INTEGER, name="group_lb", lb = 0, ub = n_levels)
    group_upper_bound = model.addVars(max_min_groups, vtype = GRB.INTEGER, name="group_ub", lb = 0, ub = n_levels)
    group_level_range = model.addVars(max_min_groups, vtype = GRB.INTEGER, name="group_range", lb = 0, ub = n_levels)
    Zu = model.addVars(max_min_groups, n_levels, vtype = GRB.BINARY, name="Zu")
    Zl = model.addVars(max_min_groups, n_levels, vtype = GRB.BINARY, name="Zl")

  # print("add elem constrs")

  for i, j in S.keys():
    model.addConstr(gp.quicksum(y[i, j, k] for k in range(len(sections))) == 1)

    for k in range(len(sections)):
        model.addConstr((y[i, j, k] == 1) >> (element_section[i, j] == k))  # Enforce correct index
        model.addConstr((y[i, j, k] == 1) >> (cost_section[i, j] == SectionCost[k]))


  for c in range(n_cols):
    for l in range(n_levels):
      #sum of Xgcl over all groups must = if col at that level exists
      model.addConstr(gp.quicksum(x[g,c,l] for g in range(max_min_groups)) == E[c,l])
      model.addConstr(element_section[c,l] >= S[c,l])
      # model.addConstr(gp.quicksum(element_section[c,l,s] for s in range(M_sections)) == 1)

    for l in range(1,n_levels):
      if E[c,l] == 1 and E[c,l-1] == 1:
        model.addConstr(element_section[c,l] <= element_section[c,l-1])

  # print("add grp constrs")
  for g in range(max_min_groups):
    print(g, g/max_min_groups)
    #range of group g = upper bound - lower bound
    model.addConstr(group_level_range[g] == group_upper_bound[g] - group_lower_bound[g])

    #1 section per group
    # model.addConstr(gp.quicksum(group_section[g] for s in range(M_sections)) == 1)

    for c in range(n_cols):
      for l in range(n_levels):

        #if element is in a group, it's section is at least the group's
        #if not in the group, it's greater than (at most) 0
        #TODO: these are problems constraints.
        model.addConstr(element_section[c,l] >= group_section[g] - M_sections*(1-x[g,c,l]))
        model.addConstr(element_section[c,l] <= group_section[g] + M_sections*(1-x[g,c,l]))

        #if element is in group, that column is in the group
        model.addConstr(column_in_group[g,c] >= x[g,c,l])

        #if element is in group, that level is in the group
        model.addConstr(level_in_group[g,l] >= x[g,c,l])

        #if element is in that group, that group exists
        model.addConstr(group_exists[g] >= x[g,c,l])

  # print("add lvl constrs")
  for g in range(max_min_groups):
    for l in range(n_levels):

      #calculate upper, lower level
      #NOTE: need to min(Ug) and max(Lg) for this to work
      #NOTE: in julia, everything is 1 based, might need to adjust formulation to make everything work as 0-based
      model.addConstr(l*level_in_group[g,l] <= group_upper_bound[g])
      model.addConstr(l*level_in_group[g,l] + M*(1-level_in_group[g,l]) >= group_lower_bound[g])

      #set Zl
      model.addConstr(M*Zl[g,l] >= l-group_lower_bound[g]+1)
      model.addConstr(M*(1-Zl[g,l]) >= group_lower_bound[g]-l)

      #set Zu
      model.addConstr(M*Zu[g,l] >= group_upper_bound[g]-l+1)
      model.addConstr(M*(1-Zu[g,l]) >= l-group_upper_bound[g])

      #if a level is within lower/upper bound, it's in the group
      model.addConstr(1+level_in_group[g,l] >= Zu[g,l]+Zl[g,l])

      # for c in range(n_cols):
      #   #if column and level are in the group, so is the element
      #   #todo: lazily constraint??
      #   model.addConstr(column_in_group[g,c]+level_in_group[g,l] <= 1 + x[g,c,l])

  model.setObjective(gp.quicksum(cost_section[i, j] for i, j in S.keys()) + GroupCost*gp.quicksum(group_exists) - gp.quicksum(group_upper_bound) + gp.quicksum(group_lower_bound),GRB.MINIMIZE)

  model.setParam('TimeLimit', 1800)
  model.setParam('SoftMemLimit', 10)

  if opt in [4,6,7,8]:
    model.Params.LazyConstraints = 1
    model.optimize(lazy_callback)
  else:
    for g in range(max_min_groups):
      for l in range(n_levels):
        for c in range(n_cols):
        #if column and level are in the group, so is the element
          model.addConstr(column_in_group[g,c]+level_in_group[g,l] <= 1 + x[g,c,l])
    model.optimize()

  # return model, x, element_section, GroupCost, column_in_group, level_in_group, max_min_groups

  # model.write("group-optim-toy.lp")

  # Optimize model

  ns = 0


  print(f"Obj: {model.ObjVal:g}")
  print(f"Time: {model.Runtime:g}")
  print("Memory Used (MiB): {}".format(round(process.memory_info().rss / 1024 ** 2,2)))

  try:

    for v in model.getVars():
      if "group_exists" in v.VarName and v.X > 0.5:
        ns += 1
    print("Groups: ", ns)


    grouped_elements = np.full((n_cols, n_levels), 0)  # -1 as default (if element doesn't exist)

    for g, i, j in x.keys():
        if x[g, i, j].X > 0.5:  # Check if x[g, i, j] is active
            grouped_elements[i, j] = g+1

    print("Grouped elements: ", grouped_elements.tolist())

    section_of_elements = np.full((n_cols, n_levels), 0)

    for c, l in element_section.keys():
        if element_section[c, l].X > 0.5:
          section_of_elements[c, l] = element_section[c, l].X

    print("Element sections: ", section_of_elements.tolist())

  except:
     print("NO SOLUTION")

  print("Original columns: ", test_data)
  print("Section costs: ", SectionCost)
  print("Group cost: ", GroupCost)




  # heatmap(grouped_elements, section_of_elements)

  print("---ALGORITHM END")

## **Results**

Given our improvements, we have a 8 different models. The following table shows which improvement has been implemented on each model.

$$
\begin{array}{|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Variation:} & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 \\
\hline
\text{Continuous Vars} &  & X &  &  & X & X &  & X \\
\text{Binary Element} &  &  & X &  & X &  & X & X \\
\text{Section Vars} &  &  &  &  &  &  &  &  \\
\text{Lazy Constraints} &  &  &  & X &  & X & X & X \\
\hline
\end{array}
$$


In the following table we can see why each of the models for each instance stopped:
$$
\begin{array}{|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Instance}  & 2 & 3 & 4 & 5 & 6 & 7 & 8 \\
\hline
\text{Mid-rise} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Mem-out} & \text{Time-out} & \text{Mem-out} & \text{Mem-out}  \\
\text{LTC sections} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Mem-out}  \\
\text{LTC levels} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} &  \text{Mem-out}\\
\text{Industrial} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out} & \text{Time-out}  \\
\hline
\end{array}
$$


# **Phase Two Improvements**

After obtaining feedback from colleagues, two new ideas were proposed: symmetry breaking constraints, and clique constraints. The following section will detail each one individiaully, and then present results from an ablation study for these improvements.

## Symmetry Breaking Constraints

### Motivation

One issue with the formulation is there is a large degree of symmetry. For example, Group 1 containing elements {(1,3),(2,3)} and Group 2 containing elements {(